In [1]:
from PIL import Image
import numpy as np
import os
from random import shuffle
import matplotlib.pyplot as plt

DIR = 'C:\Yelp\Train'

# Want to know how we should format the height x width image data dimensions
# for inputting to a keras model
def get_size_statistics():
    heights = []
    widths = []
    img_count = 0
    for img in os.listdir(DIR):
        path = os.path.join(DIR, img)
        if "DS_Store" not in path:
            data = np.array(Image.open(path))
            heights.append(data.shape[0])
            widths.append(data.shape[1])
            img_count += 1
    avg_height = sum(heights) / len(heights)
    avg_width = sum(widths) / len(widths)
    print("Average Height: " + str(avg_height))
    print("Max Height: " + str(max(heights)))
    print("Min Height: " + str(min(heights)))
    print('\n')
    print("Average Width: " + str(avg_width))
    print("Max Width: " + str(max(widths)))
    print("Min Width: " + str(min(widths)))
get_size_statistics()

Average Height: 383.958217270195
Max Height: 400
Min Height: 150


Average Width: 482.1309192200557
Max Width: 600
Min Width: 200


In [2]:
def label_img(name):
    word_label = name.split('(')[0]
    #print(word_label)
    if word_label == 'Burger ':  return 0
    elif word_label == 'pizza ': return 1
    elif word_label == 'Chicken ': return 2
    elif word_label == 'Sweet ': return 3
    elif word_label == 'Tacos ': return 4
    elif word_label == 'Drink ': return 5

In [3]:
IMG_SIZE = 256
def load_training_data():
    train_data = []
    for img in os.listdir(DIR):
        label = label_img(img)
        path = os.path.join(DIR, img)
        if "DS_Store" not in path:
            img = Image.open(path)
            img = img.convert('L')
            img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
            train_data.append([np.array(img), label])
            X_train=np.array(img)
            Y_train=label
    shuffle(train_data)
    return train_data

In [4]:
train_data = np.array(load_training_data())
y=len(train_data)
X=[]
Y=[]

In [5]:
for i in range(y-1):
    img=np.array((train_data[i][0]).reshape(256,256))
    X.append(img)
    label=np.array(train_data[i][1])
    Y.append(label)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    random_state=42,
                                                    test_size=0.20)

In [7]:
X_train=np.array(X_train)
y_train=np.array(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(286, 256, 256)
(72, 256, 256)
(286,)
(72,)


In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [11]:
classifier=Sequential()
classifier.add(Conv2D(64,3,3,input_shape=(256,256,1),activation='relu'))
classifier.add(Conv2D(32,3,3,activation='relu'))
#classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(output_dim=6,activation='softmax'))
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


D:\Python\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(256, 256,..., activation="relu")`
  
D:\Python\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Python\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=6)`
  


In [12]:
X_train = X_train.reshape(286,256,256,1)
X_test = X_test.reshape(72,256,256,1)
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [15]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 32)      18464     
_________________________________________________________________
flatten_1 (Flatten)          (None, 2032128)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 12192774  
Total params: 12,211,878
Trainable params: 12,211,878
Non-trainable params: 0
_________________________________________________________________


In [16]:
classifier.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Instructions for updating:
Use tf.cast instead.
Train on 286 samples, validate on 72 samples
Epoch 1/3
286/286 [==============================] - 102s 356ms/step - loss: 11.4991 - acc: 0.2692 - val_loss: 12.3124 - val_acc: 0.2361
Epoch 2/3
286/286 [==============================] - 101s 352ms/step - loss: 11.4405 - acc: 0.2902 - val_loss: 12.3124 - val_acc: 0.2361
Epoch 3/3
286/286 [==============================] - 97s 340ms/step - loss: 11.4405 - acc: 0.2902 - val_loss: 12.3124 - val_acc: 0.2361


In [17]:
classifier.evaluate(X_test, y_test) 

72/72 [==============================] - 7s 97ms/step


[12.312434196472168, 0.2361111111111111]

In [18]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(256,256,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(6, activation = 'sigmoid'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 254, 254, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 127, 127, 32)      128       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 62, 62, 64)        256       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 60, 60, 96)        55392     
__________

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [21]:
model.fit(X_train,y_train,epochs = 10, validation_steps=None,verbose = 1,batch_size=50)

Epoch 1/10
286/286 [==============================] - 62s 216ms/step - loss: 1.7412 - acc: 0.3147
Epoch 2/10
286/286 [==============================] - 61s 212ms/step - loss: 1.3814 - acc: 0.5769
Epoch 3/10
286/286 [==============================] - 60s 210ms/step - loss: 1.2055 - acc: 0.5874
Epoch 4/10
286/286 [==============================] - 60s 208ms/step - loss: 0.9961 - acc: 0.6294
Epoch 5/10
286/286 [==============================] - 60s 210ms/step - loss: 0.7748 - acc: 0.7063
Epoch 6/10
286/286 [==============================] - 60s 209ms/step - loss: 0.6189 - acc: 0.7448
Epoch 7/10
286/286 [==============================] - 63s 219ms/step - loss: 0.4206 - acc: 0.8357
Epoch 8/10
286/286 [==============================] - 62s 215ms/step - loss: 0.2715 - acc: 0.9091
Epoch 9/10
286/286 [==============================] - 46s 161ms/step - loss: 0.1937 - acc: 0.9580
Epoch 10/10
286/286 [==============================] - 41s 145ms/step - loss: 0.1626 - acc: 0.9580


In [22]:
model.evaluate(X_test, y_test)

72/72 [==============================] - 3s 40ms/step


[1.3441175884670682, 0.5833333333333334]

In [23]:
predictions=model.predict(X_test)

In [43]:
Pred_Max_Positions=[]
for i in range(72):
    res=max(predictions[i])
    #print(res)
    for j in range(6):
        if (res==predictions[i][j]):
            result=j
            Pred_Max_Positions.append(j)
#Pred_Max_Positions

In [46]:
Test_Max_Positions=[]
for i in range(72):
    res=max(y_test[i])
    #print(res)
    for j in range(6):
        if (res==y_test[i][j]):
            result=j
            Test_Max_Positions.append(j)
#Test_Max_Positions

In [47]:
from sklearn.metrics import confusion_matrix
CM_CNN=confusion_matrix(Test_Max_Positions,Pred_Max_Positions)
CM_CNN

array([[12,  0,  0,  0,  0,  6],
       [ 4, 18,  0,  1,  0,  0],
       [ 2,  0,  0,  0,  0,  0],
       [ 7,  1,  0,  0,  0,  3],
       [ 1,  0,  0,  0,  0,  0],
       [ 4,  0,  0,  1,  0, 12]], dtype=int64)

In [38]:
from sklearn.metrics import classification_report
print(classification_report(Test_Max_Positions,Pred_Max_Positions))

              precision    recall  f1-score   support

           0       0.40      0.67      0.50        18
           1       0.95      0.78      0.86        23
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00        11
           4       0.00      0.00      0.00         1
           5       0.57      0.71      0.63        17

   micro avg       0.58      0.58      0.58        72
   macro avg       0.32      0.36      0.33        72
weighted avg       0.54      0.58      0.55        72

